In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

# Load the data

In [2]:
data = pd.read_csv('long-method-2020+2019+2018.csv')
data = data[['LOC_method', 'CC_method','is_long_method']]

# Normalize the data

In [3]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.drop(columns='is_long_method'))

X = data_scaled.reshape(data_scaled.shape[0], data_scaled.shape[1], 1)
y = data['is_long_method'].values

# Split data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Building

In [5]:
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(2, 1)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 32)             96        
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 128)               4224      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,641
Trainable params: 12,641
Non-traina

# Train the model

In [6]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
34/34 [==============================] - 3s 24ms/step - loss: 0.6817 - accuracy: 0.6170 - val_loss: 0.6704 - val_accuracy: 0.5902
Epoch 2/10
34/34 [==============================] - 0s 11ms/step - loss: 0.6578 - accuracy: 0.6509 - val_loss: 0.6471 - val_accuracy: 0.6541
Epoch 3/10
34/34 [==============================] - 0s 8ms/step - loss: 0.6184 - accuracy: 0.6745 - val_loss: 0.6161 - val_accuracy: 0.6767
Epoch 4/10
34/34 [==============================] - 0s 7ms/step - loss: 0.5837 - accuracy: 0.7123 - val_loss: 0.5818 - val_accuracy: 0.7068
Epoch 5/10
34/34 [==============================] - 0s 7ms/step - loss: 0.5626 - accuracy: 0.7245 - val_loss: 0.5520 - val_accuracy: 0.7556
Epoch 6/10
34/34 [==============================] - 0s 7ms/step - loss: 0.5292 - accuracy: 0.7632 - val_loss: 0.5387 - val_accuracy: 0.7744
Epoch 7/10
34/34 [==============================] - 0s 7ms/step - loss: 0.5225 - accuracy: 0.7575 - val_loss: 0.5209 - val_accuracy: 0.7594
Epoch 8/10
34/34 [

# Evaluation

In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("\nTest accuracy:", test_acc)

9/9 - 0s - loss: 0.5032 - accuracy: 0.7707 - 84ms/epoch - 9ms/step

Test accuracy: 0.7706766724586487


In [8]:
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import f1_score
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")

9/9 [==============================] - 0s 5ms/step
Recall: 0.8055555555555556
Precision: 0.6850393700787402
F1 Score: 0.7404255319148937


In [ ]:
import joblib
model.save('lm_model.h5')
joblib.dump(scaler, 'lm_scalar.pkl')